In [1]:
import sys
import numpy as np

import pandas as pd
import warnings
import matplotlib.pyplot as plt
import seaborn as sns
warnings.filterwarnings('ignore')
%matplotlib inline
%config InlineBackend.figure_format = 'svg'
plt.style.use('ggplot')
plt.rcParams["font.family"] = "Times New Roman"

In [2]:
import torch
import torchvision
from torchvision import models
from torch import nn
import torch.nn.functional as F
from torch import optim
from tqdm import tqdm_notebook
import torchvision.transforms as T
from sklearn.model_selection import train_test_split
import time
import cv2
from PIL import Image
from sklearn.metrics import mean_squared_error as mse
import timm
import tez
import os
import glob

In [3]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(device)

In [5]:
PATH = '/kaggle/input/petfinder-pawpularity-score/'
IMG_SIZE = 384
BATCH_SIZE = 12
model_name = 'swin_large_patch4_window12_384'

In [6]:
df_train = pd.read_csv('../input/train-data-10/10FOLDSTRAIN.csv')
df_test = pd.read_csv(PATH + 'test.csv')
df_train.head(2)

In [7]:
# *************************************************
FOLD = 1
# *************************************************

In [8]:
X_train = df_train.loc[df_train['kfold'] != FOLD]
X_valid = df_train.loc[df_train['kfold'] == FOLD]
print(len(X_train), len(X_valid))

In [9]:
augm_train = T.Compose([T.ToPILImage(),
#                         T.RandomHorizontalFlip(p=0.2),
                        T.Resize([IMG_SIZE, IMG_SIZE]),
                        T.ToTensor(),
                        T.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
                       ])

augm_test = T.Compose([T.ToPILImage(),
                       T.Resize([IMG_SIZE, IMG_SIZE]),
                       T.ToTensor(),
                       T.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
                      ])

In [10]:
class PawDataset(torch.utils.data.Dataset):
    def __init__(self, df, image_path='train/', augm=False):
        self.df = df.copy()
        self.augm = augm
        self.img_path = PATH + image_path


    def __len__(self):
        return len(self.df)

    def __getitem__(self, idx):
        image = cv2.imread(self.img_path + str(self.df.iloc[idx, 0]) + '.jpg')
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        image = self.augm(image)

        target = self.df.iloc[idx, -1:]
        features = torch.FloatTensor(self.df.iloc[idx, 1:-1].values.astype('float32'))
        target = torch.FloatTensor(target.values.astype('float32'))
    

        return image, features, target/100

In [11]:
train_data = PawDataset(X_train.drop(columns='kfold'), augm=augm_train)
valid_data = PawDataset(X_valid.drop(columns='kfold'), augm=augm_test)
test_data = PawDataset(df_test, image_path='test/', augm=augm_test)

In [12]:
train_loader = torch.utils.data.DataLoader(train_data, batch_size=BATCH_SIZE, shuffle=True, drop_last=True, num_workers=8)
valid_loader = torch.utils.data.DataLoader(valid_data, batch_size=BATCH_SIZE*2, shuffle=False, num_workers=8)
test_loader = torch.utils.data.DataLoader(test_data, batch_size=BATCH_SIZE*2, shuffle=False, num_workers=8)

In [13]:
model_ = timm.create_model(model_name, pretrained=True, in_chans=3)

In [14]:
# model_ = torch.nn.Sequential(*(list(model.children())[:-1]))

In [15]:
for param in model_.parameters():
    param.requires_grad = False
model_.head = nn.Linear(1536, 128)
model_.head.requires_grad = True

In [16]:
class MyNet(tez.Model):
    def __init__(self, base_model):
        super().__init__()
        self.model = base_model
        self.dropout1 = nn.Dropout(0.5)
        self.dropout2 = nn.Dropout(0.1)
        self.dense1 = nn.Linear(128, 1)


    def forward(self, image, features, targets=None):
        x = self.model(image)
        x = self.dropout2(x)
        x = F.relu(x)
        x = self.dense1(x)
        return x

net = MyNet(base_model=model_)
net.to(device)

In [17]:
# net2 = torch.load('../input/lastmod/only_img_model1836.pht')
# print(net2)

In [18]:
LR = 0.0005
EPOCHES = 3
criterion  = nn.BCEWithLogitsLoss()
opt = optim.Adam(net.parameters(), lr=LR)
scheduler = optim.lr_scheduler.MultiStepLR(opt, milestones=[1,2,3], gamma=0.5)

In [19]:
def sigmoid(x):
    return 1 / (1 + np.exp(-x))

def rmse_func(y_pred, y_true):
    y_ = 100 * y_true
    pred_ =  sigmoid(y_pred)*100
    return np.sqrt(mse(y_, pred_))

In [20]:
def save_model(model, mod_path, prefix, metric, verbose=True):
    path_1 = mod_path + prefix   
    for f in glob.glob(path_1+"*"):
        os.remove(f)
    full_path = path_1 + str(round(metric*100)) +'.pht'
    torch.save(model, full_path)
    if verbose:
        print(f'Model saved with metric: {round(metric,2)}')

In [21]:
def model_eval(model, opt, loss_func, dataset_loader):

    num_batches = len(dataset_loader)
    test_loss = 0
    rmse = 0
    full_y = np.array([])
    full_pred = np.array([])
    with torch.no_grad():
        model.eval()
        for X, features, y in dataset_loader:
            pred = model(X.to(device), features.to(device))
            test_loss += criterion(pred, y.to(device)).item()
            full_y = np.append(full_y, y.cpu().numpy())
            full_pred = np.append(full_pred, pred.cpu().numpy())


        test_loss = round(test_loss / num_batches, 4)
        rmse = rmse_func(full_pred, full_y)
        model.train()
        

    return test_loss, rmse

In [22]:
def train_step(epoch, train_start_time, model, opt, loss_func, train_loader, step, best_metric):

    time_start = time.time()
    running_loss = 0
    num_batches_train = len(train_loader)

    for num, data in enumerate(train_loader):
        images, features, labels = data[0].to(device), data[1].to(device), data[2].to(device)

        opt.zero_grad()  # обнуляем градиент
        outputs = model(images, features)  # получаем предсказания
        loss = loss_func(outputs, labels)  # считаем потери
        loss.backward()  # ОРО
        opt.step()  # обновление весов

        running_loss += loss.item()

        if (num+1) % step == 0:
            test_loss, rmse = model_eval(model, opt, loss_func, valid_loader)
#             save_model(net, mod_path='./' , prefix='only_img_model', metric=rmse)
            print(
            f'IT № {num+1}/{num_batches_train}, tr_loss= {round(running_loss/(num+1), 4)}, eval_loss= {test_loss},',
            f'rmse = {round(rmse,3)}'
        )
            if rmse < best_metric:
                best_metric = rmse
                save_model(model, mod_path='./', prefix=f'only_img_model_{FOLD}_', metric=best_metric)

    test_loss, rmse = model_eval(model, opt, loss_func, valid_loader)
    train_loss = round(running_loss / num_batches_train, 4)

    if (epoch+1) % 1 == 0:
        time_taken = round(time.time() - time_start, 1)
        from_start = round(time.time() - train_start_time, 1)
        print(
            f'Epoch № {epoch+1}, tr_loss= {train_loss}, eval_loss= {test_loss},',
            f'rmse = {rmse}',
            f'\ntime_per_epoch = {time_taken} sec, total ---> {from_start} sec'
        )
    return rmse, best_metric

In [23]:
train_start = time.time()
rmse_history = []
best_metric = 1000
step = 200
for epoch in tqdm_notebook(range(EPOCHES)):
    print('*'*40, f'EPOCH № {epoch+1}', '*'*40)
    rmse, best_metric = train_step(epoch,
                      train_start,
                      net,
                      opt,
                      criterion,
                      train_loader,
                      step,
                      best_metric
                     )

        
    if best_metric < 19:
        step = 40
        if best_metric < 18.4:
            step = 20
            if (best_metric < 18.1) or (epoch >= 1):
                step = 15
            
        
        
    rmse_history.append(rmse)
    scheduler.step()
print('*'*100)
print(f'Training is finished! Best score: {min(rmse_history)} Time taken, sec:',
      time.time()-train_start)